In [ ]:
from parse_tree import Node,Graph,Digraph,nohtml,Source
import nltk

#Expr class to store each expression
class Expr(object):
    def __init__(self,i,a,sigma,tau,j,method):
        self.i = i
        self.a = a
        self.sigma = sigma
        self.method = method
        self.tau = tau
        self.j = j
    def __str__(self):
        return 's[' + str(self.i) + ']: ' + self.a + ' → ' + self.sigma + '•' + self.tau + ', ' + str(self.j)
    
#define an empty list to store the data used for animation
graph_data = []

#### Modification version of earley's parser from Lecture notes 11
Purpose: To return a parse tree in certain form   
         Each brackets represents a leaf or tree.

In Match step, instead of adding `(A → σ a • ω, j)`, we add `(A → σ (a) • ω, j)`   
In Complete step, instead of adding `(B → μ A • ξ, k)`, we add `(B → μ (Aσ) • ξ, k)`

In [ ]:
def parse(g: "grammar", x: "input"):
    global s
    n = len(x); x = '^' + x + '$'; S, π = g[0][0], g[0][2:]
    s = [{(S, '', π, 0)}] + [set() for _ in range(n)]#; print('   s[ 0 ]:', S, '→ •', π, ', 0')
    for i in range(n + 1):
        v = set() # visited items
        while v != s[i]:
            e = (s[i] - v).pop(); v.add(e) # pick an arbirary un-visited item
            A, σ, τ, j = e
            if len(τ) > 0 and τ[0] == x[i + 1]: # match, a == τ[0]
                f = (A, σ + '(' + τ[0] + ')', τ[1:], j)
                s[i + 1].add(f)
                print('M  s[', i + 1, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
                #store animation data
                graph_data.append(Expr(i+1, f[0], f[1], f[2], f[3], "Match"))
            elif len(τ) > 0: # predict, B == ω[0]
                for f in ((r[0], '', r[2:], i) for r in g if r[0] == τ[0]):
                    s[i].add(f)
                    print('P  s[', i, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
                    #store animation data
                    graph_data.append(Expr(i, f[0], f[1], f[2], f[3], "Predict"))
            else: # complete, len(τ) == 0
                for f in ((B, μ + '(' + ν[0] + σ + ')', ν[1:], k) for (B, μ, ν, k) in s[j] if len(ν) > 0 and ν[0] == A):
                    s[i].add(f)
                    print('C  s[', i, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
                    #store animation data
                    graph_data.append(Expr(i, f[0], f[1], f[2], f[3], "Complete"))
    return {f[1] for f in s[n] if f[0] == S}

In [ ]:
G = ("S→E", "E→a", "E→a+E")
parse(G, "a+a+a")

In [ ]:
#generate png image file for each step
#Expr(i,a,sigma,tau,j,method)
for i in range(len(graph_data)):
    t_str = '('+graph_data[i].a+graph_data[i].sigma+')'
    t = nltk.Tree.fromstring(t_str)
    src = Graph(t).draw()
    g = Source(src,filename=str(i))
    g.render(format='png')

In [ ]:
from ipywidgets import *
from IPython.display import Image
import time

#set this to TRUE to auto-generate the stages
auto_generate = False

#define a button
btn = widgets.Button(description='next')
display(btn)

#display generated png image using interactive button
class Animate(object):
    def __init__(self,data):
        self.graph_data = data
        self.i = 0
        
    def func(self,btn):
        if self.i < len(graph_data):
            item = graph_data[self.i]
            print("\x1b[91m"+item.method+"\x1b[0m")
            print(item)
            display(Image(filename=str(self.i)+'.png'))
            self.i += 1
        else:
            print("===FINISH===")
            
#initiate Animate class
anime = Animate(graph_data)
if auto_generate:
    for i in range(len(graph_data)):
        time.sleep(1)
        anime.func(btn)
else:
    btn.on_click(anime.func)